In [1]:
import pandas as pd 
import numpy as np
from activitysim.core import inject
from activitysim.core.util import reindex

In [2]:
pd.options.mode.chained_assignment = None 

In [59]:
year = 2030
scenario = 'core'

inject.add_injectable('year', year)
inject.add_injectable('scenario', scenario)

In [96]:
## What it should be available at: 
households = pd.read_csv('../bay_area_base/data/households.csv')
persons = pd.read_csv('../bay_area_base/data/persons.csv').sample(10000)

households = households.set_index('household_id')
persons = persons.set_index('person_id')

households['hh_value_of_time'] = np.random.rand(len(households))
# persons['vot'] = np.random.rand(len(persons))

In [97]:
persons_merge = persons.merge(households, how = 'inner', left_on = 'household_id', right_index = True )

In [3]:
av_rates = pd.DataFrame({'conservative':[0, 0.1, 0.15, 0.20, 0.3, 0.4, 0.25], 
              'core':[0, 0.3, 0.4, 0.5, 0.6, 0.7, 0.4],
              'aggressive':[0, 0.5, 0.65, 0.80, 0.9, 0.95, 0.6]}, 
             index = [2025, 2030, 2035, 2040, 2045, 2050, "vot_reduction"])

In [5]:
av_rates.to_csv('../../bay_area/configs/av_rates.csv')

In [99]:
year = inject.get_injectable('year')
scenario = inject.get_injectable('scenario')

rate = av_rates.loc[year, scenario]
vot_reduction = av_rates.loc['vot_reduction',scenario]

In [100]:
av = np.random.choice(np.array([0,1]), size = len(households), p = np.array([1 - rate, rate]))

In [101]:
households['av'] = av
households['hh_value_of_time'] = households['hh_value_of_time'].mask(households.av.astype(bool), 
                                                                     households['hh_value_of_time']*(1 - vot_reduction))

In [103]:
persons['_hh_vot'] = reindex(households.hh_value_of_time, persons.household_id)

In [105]:
persons['value_of_time'] = persons['_hh_vot'].where(persons.age>=18, persons._hh_vot * 0.667)

In [113]:
persons_merge['value_of_time'] = persons.value_of_time

In [114]:
persons_merge

,age,earning,edu,hispanic,hours,member_id,race_id,relate,sex,student,...,block_id,lcm_county_id,persons,gt55,gt2,hh_type,TAZ_y,HHT,hh_value_of_time,value_of_time
person_id,,,,,,,,,,,,,,,,,,,,,
2936469,60,100000.0,22.0,0,40.0,2,1,1,2,0,...,60750227042002,6075,2,1,1,4,114,4,0.634117,0.634117
2766003,44,110000.0,20.0,0,50.0,2,1,1,1,0,...,60552017002000,6055,4,0,1,3,1313,4,0.963877,0.963877
6240974,63,54000.0,16.0,0,36.0,2,1,1,2,0,...,60952508013002,6095,2,1,1,8,1222,4,0.645154,0.645154
689638,19,0.0,18.0,0,0.0,4,1,2,1,1,...,60014352001032,6001,4,0,1,3,843,4,0.752465,0.752465
1962964,43,0.0,19.0,0,0.0,1,1,0,1,0,...,60133553044020,6013,1,0,0,5,1100,1,0.615479,0.615479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369253,34,55000.0,23.0,0,40.0,1,1,0,2,1,...,60014225003010,6001,1,0,0,5,1020,1,0.926262,0.926262
1298106,84,0.0,18.0,0,0.0,1,1,0,2,0,...,60014506071026,6001,1,1,0,6,739,1,0.130867,0.130867
4332430,56,15000.0,15.0,0,30.0,1,6,0,1,0,...,60855033153018,6085,3,1,1,8,650,4,0.661203,0.661203


In [32]:
rate = 0.2 
vot_reduction = 0.25
av = np.random.choice(np.array([0,1]), size = len(persons), p = np.array([1 - rate, rate]))

In [33]:
persons['av'] = av
persons['vot'] = persons.vot.mask(persons.av.astype(bool), persons.vot*(1 - vot_reduction))

In [38]:
persons['vot'] = persons.vot.mask(persons.av.astype(bool), persons.vot*(1 - vot_reduction))

In [39]:
persons

,age,earning,edu,hispanic,hours,member_id,race_id,relate,sex,student,...,TAZ,ptype,pemploy,pstudent,home_x,home_y,vot,car_ownership,av,vot_reduced
person_id,,,,,,,,,,,,,,,,,,,,,
39576,36,36000.0,20.0,0,60.0,1,2,0,2,0,...,978,1,1,3,-122.271245,37.824091,0.741736,0,0,0.741736
3880830,16,0.0,13.0,1,0.0,2,1,2,1,1,...,320,6,3,1,-122.237654,37.467934,0.916502,2,0,0.916502
3681015,65,10700.0,21.0,0,12.0,1,1,0,2,0,...,246,1,1,3,-122.388326,37.571410,0.738123,2,1,0.738123
3872968,31,24000.0,16.0,1,40.0,6,1,5,2,0,...,325,1,1,3,-122.233768,37.480559,0.300813,1,1,0.300813
5995386,23,2500.0,19.0,0,6.0,3,9,2,1,1,...,1270,3,1,2,-121.965096,38.279570,0.441467,0,0,0.441467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5763227,71,0.0,21.0,0,0.0,1,1,0,1,0,...,447,5,3,3,-122.009705,37.352181,0.090822,1,0,0.090822
4996653,6,0.0,4.0,0,0.0,4,2,3,2,1,...,472,7,4,1,-121.975444,37.309415,0.002408,3,0,0.002408
2192392,52,800.0,16.0,1,8.0,2,1,1,2,0,...,1179,1,1,3,-121.687007,37.926504,0.502687,0,0,0.502687


In [52]:
av_rates = pd.DataFrame({'conservative':[0, 0.1, 0.15, 0.20, 0.3, 0.4, 0.25], 
              'core':[0, 0.3, 0.4, 0.5, 0.6, 0.7, 0.4],
              'aggressive':[0, 0.5, 0.65, 0.80, 0.9, 0.95, 0.6]}, 
             index = [2025, 2030, 2035, 2040, 2045, 2050, "vot_reduction"])

In [53]:
av_rates

,conservative,core,aggressive
2025,0.00,0.0,0.00
2030,0.10,0.3,0.50
2035,0.15,0.4,0.65
2040,0.20,0.5,0.80
2045,0.30,0.6,0.90
2050,0.40,0.7,0.95
vot_reduction,0.25,0.4,0.60


In [48]:
year = inject.get_injectable('year')
scenario = inject.get_injectable('scenario')

rate = av_rates.loc[year, scenario]
vot_reduction = av_rates.loc['vot_reduction',scenario]

In [54]:
rate = av_rates.loc[year, scenario]
vot_reduction = av_rates.loc['vot_reduction',scenario]

In [55]:
rate

0.3

In [56]:
vot_reduction

0.4

In [57]:
households['vot'] = 

,serialno,cars,income,race_of_head,age_of_head,workers,hispanic_status_of_head,tenure,recent_mover,sf_detached,...,tenure_mover,hh_seniors,block_id,lcm_county_id,persons,gt55,gt2,hh_type,TAZ,HHT
household_id,,,,,,,,,,,,,,,,,,,,,
0,2013000040996,1.0,116030.0,6,61,1.0,0,1,0,yes,...,own not recent,no,60014001001024,6001,1,1,0,2,1005,1
1,2011000868263,1.0,110000.0,2,51,1.0,0,1,0,no,...,own not recent,no,60014001001051,6001,1,0,0,1,1005,1
2,2011000751406,1.0,131000.0,1,64,0.0,0,1,0,yes,...,own not recent,no,60014001001058,6001,1,1,0,2,1005,1
3,2009001230781,1.0,134000.0,1,53,1.0,0,1,0,no,...,own not recent,no,60014001001027,6001,1,0,0,1,1005,1
4,2009001136328,1.0,140400.0,1,62,1.0,0,1,0,no,...,own not recent,no,60014001001074,6001,1,1,0,2,1005,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2608018,2010000870502,2.0,23700.0,1,65,2.0,0,2,0,yes,...,rent not recent,yes,60971543043072,6097,2,1,1,8,1403,4
2608019,2012001297125,5.0,59000.0,1,63,2.0,0,1,0,yes,...,own not recent,yes,60971543043072,6097,2,1,1,4,1403,4
2608020,2010000362541,1.0,109000.0,1,71,1.0,0,2,0,no,...,rent not recent,yes,60971543043013,6097,1,1,0,6,1403,1


In [116]:
av_rates.loc[2020, 'conservative']

KeyError: 2020